In [28]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import warnings
from models.interactive_parameter import InteractiveUtils

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

**Generalised logistic function (GLF)**

![y = A + \frac{K - A}{\left( C + Q \cdot e^{-By} \right)^{ 1 / \nu} }](assets/GLF.png)

taken from https://en.wikipedia.org/wiki/Generalised_logistic_function

**Inverse GLF**
The following equation is the one used in this simulation.

![y = \frac{1}{B} \cdot ln \left( \frac{ \left( \frac{K - A}{x - A} \right)^\nu -C }{Q} \right) + E](assets/GLF_inverse.png)

A bunch of parameter can be removed, as they are not used:

- `A` is the minimum value of the x axis, which will always be `0`
- `C` and `Q` are form parameters that are meant to be set to `1` by default

With those parameters replaced by their default values, here is the simplified equation used:

![y = \frac{1}{B} \cdot ln \left( \left( \frac{K}{x} \right)^\nu -1 \right) + E](assets/GLF_inverse_simplified.png)

This equation can be generalised in other dimensions as follow:

![y = E + \prod_{i=1}^n \sqrt[n]{B_i \cdot ln \left( \left( \frac{K_i}{x_i} \right)^{\nu_i} -1 \right) + E_i}](assets/GLF_inverse_simplified_generalised.png)


In [31]:
from matplotlib.axes import Axes


def clip(
    values: np.ndarray, min_value: float = None, max_value: float = None
) -> np.ndarray:
    return np.clip(values, min_value, max_value)


def equation_single(X, num, *args, **kwargs):
    B = kwargs[f"B{num}"]
    K = kwargs[f"K{num}"]
    nu = kwargs[f"nu{num}"]
    E = kwargs[f"E{num}"]

    try:
        inner = np.power((K / X), nu) - 1
        output: np.ndarray = clip(E + B * np.log(inner), 0, 8.5)
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)

    return output


def equation_multi(X, Y, E, E1, B1, K1, nu1, E2, B2, K2, nu2):
    try:
        inner1 = np.power((K1 / X), nu1) - 1
        inner2 = np.power((K2 / Y), nu2) - 1

        part1 = np.power(E1 + B1 * np.log(inner1), 1 / 2)
        part2 = np.power(E2 + B2 * np.log(inner2), 1 / 2)

        part1 = np.nan_to_num(part1, posinf=0)
        part2 = np.nan_to_num(part2, posinf=0)

        output = clip(
            clip(part1, 0) * clip(part2, 0) + E,
            0,
            8.5,
        )

    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)  # * np.nan

    # replace nans by 0
    output = np.nan_to_num(output)

    return output


def sigmoid_plot(
    ax: Axes, T, grid, x, color=None, label=None, alpha=None, theta=None, only_3d=False
):
    if isinstance(x, tuple):
        ax.plot_wireframe(
            *grid,
            T(*grid),
            linewidth=1,
            cmap=cm.coolwarm,
            rstride=50,
            cstride=50,
            antialiased=True,
            alpha=1,
        )

        if not only_3d:
            ax.text2D(
                0.5,
                0.85,
                f"APR: {T(*x):.2f}%",
                transform=ax.transAxes,
                horizontalalignment="center",
                bbox=dict(facecolor="red", alpha=0.5),
            )

        ax.set_xlabel(label[0])
        ax.set_ylabel(label[1])
        ax.set_zlabel("APR (%)")

        ax.view_init(alpha, theta)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.grid(which="minor")

        ax.set_zlim(0, 10)
        ax.set_box_aspect(aspect=None, zoom=0.8)

    else:
        ax.plot(grid, T(grid), label="X", color=color)
        ax.vlines(x, 0, T(x), color="k", linestyle="dashed")
        ax.hlines(T(x), -0.1, x, color="k", linestyle="dashed")

        ax.text(
            0.5,
            0.95,
            f"APR: { T(x):.2f}%",
            transform=ax.transAxes,
            horizontalalignment="center",
            bbox=dict(facecolor="red", alpha=0.5),
        )

        ax.set_xlabel(label)
        ax.set_ylabel("APR (%)")
        # ax.set_xlim(-0.2, 0.45)
        ax.set_ylim(-5, 20)
        ax.grid()


def plot_static(
    alpha,
    theta,
    node_count,
    max_node_count,
    staked_tokens,
    max_token_supply,
    E,
    E1,
    B1,
    K1,
    nu1,
    E2,
    B2,
    K2,
    nu2,
):
    args = dict(locals())
    to_remove = [
        "alpha",
        "theta",
        "node_count",
        "max_node_count",
        "staked_tokens",
        "max_token_supply",
    ]
    for key in to_remove:
        del args[key]

    token_ratio = staked_tokens / max_token_supply
    node_ratio = node_count / max_node_count

    range_x = np.arange(0.001, 0.5, 0.001)
    range_y = np.arange(0.001, 1, 0.001)
    grid = np.meshgrid(range_x, range_y)

    fig_2d = plt.figure(figsize=(15, 4))
    fig_3d = plt.figure(figsize=(7, 7))
    axe_3d = fig_3d.add_subplot(1, 1, 1, projection="3d")
    axes_2d = [fig_2d.add_subplot(1, 2, 1), fig_2d.add_subplot(1, 2, 2)]

    # 3D plot
    sigmoid_plot(
        axe_3d,
        lambda *g: equation_multi(*g, **args),
        grid,
        (token_ratio, node_ratio),
        alpha=alpha,
        theta=theta,
        label=("Staking", "Network size"),
        only_3d=False,
    )

    # 2D plot
    sigmoid_plot(
        axes_2d[0],
        lambda r: equation_single(r, 1, **args),
        range_x,
        token_ratio,
        color="blue",
        label="Staking",
    )

    sigmoid_plot(
        axes_2d[1],
        lambda r: equation_single(r, 2, **args),
        range_y,
        node_ratio,
        color="red",
        label=f"Network size",
    )

    plt.subplots_adjust(wspace=0.2)

### Choice of base parameter for APR calculation

Although the APR could be a variable of many network parameters, those which makes most sense to use are the following ones.

**Economic security ratio**

This ratio is calculated as the value of token staked in the network divided by the market capitalisation. If the staking ratio increases, the APR decreases.
It's a gamable paramter: if a whale comes in the network and put a huge stake in his nodes' safe(s), he would increase this ratio drastically, thus would reduce the overall APR.

**Network size**

Eventhough we want the network to be as big as possible at somepoint, we don't want that this amount grows to fast. If the network size increases, the APR decreases. This is already a side effect of the current fix-budget, and should be kept in this new incentive scheme.
The network size is converted to a ratio, based on some target network size. The APR decreases with the network size increasing. This parameter prevents whales spinning up many nodes to get maximum rewards from their stake.


In [32]:
params = InteractiveUtils.loadSimulationConfigFile("assets/sigmoid.config.pow.1.2.yaml")
InteractiveUtils.plotInteractive(plot_static, params)

Output()